# AIPI 590 - XAI | Assignment #1
### Interpretable ML Assignment
### Lindsay Gross


[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lindsaygross/Interpretable-ML/blob/lindsay_assign1/Interpretable_ML.ipynb)


👉 Make sure to delete the helper markdown below in your own notebook!

## DO:
* Use markdown and comments effectively
* Pull out classes and functions into scripts 
* Ensure cells are executed in order and avoid skipping cells to maintain reproducibility
* Choose the appropriate runtime (i.e. GPU) if needed
* If you are using a dataset that is too large to put in your GitHub repository, you must either pull it in via Hugging Face Datasets or put it in an S3 bucket and use boto3 to pull from there. 
* Use versioning on all installs (ie pandas==1.3.0) to ensure consistency across versions
* Implement error handling where appropriate

## DON'T:
* Absolutely NO sending us Google Drive links or zip files with data (see above).
* Load packages throughout the notebook. Please load all packages in the first code cell in your notebook.
* Add API keys or tokens directly to your notebook!!!! EVER!!!
* Include cells that you used for testing or debugging. Delete these before submission
* Have errors rendered in your notebook. Fix errors prior to submission.

In [1]:
# Please use this to connect your GitHub repository to your Google Colab notebook
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Interpretable-ML" # Change to your repo name
git_path = 'https://github.com/lindsaygross/Interpretable-ML.git' #Change to your path
!git clone "/Users/lindsaygross/ME AIPI Classes/XAI/Interpretable-ML"

# Install dependencies from requirements.txt file
#!pip install -r "{os.path.join(repo_name,'requirements.txt')}" #Add if using requirements.txt
!pip install -r "{os.path.join('Interpretable-ML','requirements.txt')}" #Add if using requirements.txt


# Change working directory to location of notebook
notebook_dir = 'Interpretable_ML.ipynb' #Change to your notebook directory
path_to_notebook = os.path.join('Interpretable-ML',notebook_dir)
%cd "{path_to_notebook}"
%ls

rm: ./sample_data: No such file or directory
Cloning into 'Interpretable-ML'...
done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'Interpretable-ML/requirements.txt'
[Errno 20] Not a directory: 'Interpretable-ML/Interpretable_ML.ipynb'
/Users/lindsaygross/ME AIPI Classes/XAI/Interpretable-ML
Interpretable-ML/       Interpretable_ML.ipynb  README.md


/Users/lindsaygross/AIPI503/data-pipeline/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


#### Using environment variables in Google Colab

In Google Colab, locate the key button on the left side of the screen. You can enter in any environment variables or API keys here and they will remain private.

Then add the environment variables to your notebook using the code below.

Make sure to document in your notebook if you are using API keys and how to get them (ie share a link for documentation to get a Hugging Face API Key if you are using Hugging Face)

In [ ]:
from google.colab import userdata
userdata.get('secretName')